## 영상필터처리

In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("C:\\Users\\user\\Pictures\\public\\ml_sample.jpg")

img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
blurred = cv2.GaussianBlur(img, (7,7), 0)
edge = cv2.Canny(blurred, 50, 150)

plt.subplot(1,2,1)
plt.imshow(img, cmap = "gray")
plt.title("ori")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(edge, cmap = "gray")
plt.title("edge")
plt.axis("off")

plt.show()

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
import cv2
import numpy as np
from sklearn.svm import OneClassSVM

video_path = "C:\\Users\\user\\Pictures\\public\\old_scada.mp4"
cap = cv2.VideoCapture(video_path)


def mask_time(frame):
    frame[54:94,433:445] = 0 # 검은색,  cv2에서는 y 와 x 가 거꾸로 
    frame[298:315,122:310] = 0 # 검은색, cv2에서는 y 와 x 가 거꾸로 
    return frame

def extract_feature(frame):
    frame = mask_time(frame)
    blurred = cv2.GaussianBlur(frame, (10, 10,7),0)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 그레이스케일 , 성능을 위해서 중요, 체널 1개
    resized = cv2.resize(gray,(32, 32)) # 크기축소 (전체구조를 요약) 320 X 446 -> 32 X 32 로 바꿔서 속도향상, 메모리 절감
    feature = resized.flatten() / 255.0 # 정규화 백터화
    return feature

train_feature = []
test_feature = []
frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret: break

    feature = extract_feature(frame)

    if frame_idx == 313:
        plt.imshow(dege, cmap = "gray")
        plt.rangew(dege, cmap = "gray")
        plt.imshow(dege, cmap = "gray")
        
    if frame_idx <= 828:
        train_feature.append(feature)
    elif 828 < frame_idx < 1337:
        test_feature.append(feature)     
    frame_idx += 1 
cap.release()
X_train = np.array(train_feature)
X_test = np.array(test_feature)

# One - Class SVM 학습
ocsvm = OneClassSVM(kernel="rbf", gamma="auto", nu=0.05)
# linear , poly, sigmoid , 
# rbf: 비선형, 가우시안, 대부분 사용, 기본값
# gamma="auto" 없애면 기본값 'scale'
# nu = 이상치 값 (0 ~ 1)

# 훈련
ocsvm.fit(X_train) 

# 이상 탐지
pred = ocsvm.predict(X_test) # 1:정상 , -1: 비정상
abnormal_idx = np.where(pred==-1)[0] + 828
print("NG(공정 B의 의심)프레임:", abnormal_idx, len(abnormal_idx) )
